# Our data is present in form of directories and sub-directories...and this block will extract face from each image and then convert that data into a single numpy array and store in a compressed file.

In [2]:
from os import listdir
from os.path import isdir
import numpy as np;
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN
from PIL import Image
import cv2;

detector = MTCNN()

# extract a single face from a given photograph
def extract_face(filename, required_size=(160, 160)):
     image = Image.open(filename)
     image = image.convert('RGB')
     pixels = asarray(image)
     results = detector.detect_faces(pixels)
     if len(results)==0:
        return (False,None);
     x1, y1, width, height = results[0]['box']
     x1, y1 = abs(x1), abs(y1)
     x2, y2 = x1 + width, y1 + height
     face = pixels[y1:y2, x1:x2]
     image = Image.fromarray(face)
     image = image.resize(required_size)
     face_array = asarray(image)
     return (True,face_array);
 
    
# load images and extract faces for all images in a directory
def load_faces(directory):
     faces = [];
     if(directory.find('.DS_Store')!=-1):
      return faces;
     for filename in listdir(directory):
         if(filename.find('.DS_Store')!=-1): # if it finds a match
            continue;
         path = directory + filename
         flag,face = extract_face(path)
         if(flag):
             faces.append(face)
     return faces
 
# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
     X, y = [],[]
     for subdir in listdir(directory):
         path = directory + '/'+ subdir + '/'
         faces = load_faces(path)
         labels = [subdir for _ in range(len(faces))]
         print(labels);
         X.extend(faces)
         y.extend(labels)
     return asarray(X), asarray(y)

trainX, trainy = load_dataset('Face Dataset/train/')
print(trainX.shape, trainy.shape)
testX, testy = load_dataset('Face Dataset/test/')
savez_compressed('Face Dataset.npz', trainX, trainy, testX, testy)

['Abhishek rollno-1', 'Abhishek rollno-1', 'Abhishek rollno-1']
['Neil Dahiya', 'Neil Dahiya', 'Neil Dahiya', 'Neil Dahiya', 'Neil Dahiya']
['Pankaj Poply', 'Pankaj Poply', 'Pankaj Poply', 'Pankaj Poply', 'Pankaj Poply']
['Kushal Kumar Mehra', 'Kushal Kumar Mehra', 'Kushal Kumar Mehra', 'Kushal Kumar Mehra']
[]
['Sourav Singh Patna', 'Sourav Singh Patna', 'Sourav Singh Patna', 'Sourav Singh Patna', 'Sourav Singh Patna']
['Himanshu Judge', 'Himanshu Judge', 'Himanshu Judge', 'Himanshu Judge', 'Himanshu Judge']
['Pawan', 'Pawan', 'Pawan']
['Kamesh', 'Kamesh', 'Kamesh', 'Kamesh', 'Kamesh']
['Krishan', 'Krishan', 'Krishan', 'Krishan', 'Krishan']
['Divyansh Gupta', 'Divyansh Gupta', 'Divyansh Gupta', 'Divyansh Gupta', 'Divyansh Gupta']
['Nishant Panipat', 'Nishant Panipat', 'Nishant Panipat', 'Nishant Panipat', 'Nishant Panipat']
['Rahul Jangra', 'Rahul Jangra', 'Rahul Jangra', 'Rahul Jangra', 'Rahul Jangra']
['Avi Vinayak', 'Avi Vinayak', 'Avi Vinayak', 'Avi Vinayak', 'Avi Vinayak']
['Jati

In [3]:
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras_facenet import FaceNet
from keras.models import load_model

    
def get_embedding(model, face_pixels):
 face_pixels = face_pixels.astype('float32')
#  mean, std = face_pixels.mean(), face_pixels.std()
#  face_pixels = (face_pixels - mean) / std
 face_pixels=face_pixels.reshape(1,160,160,3);
 yhat = model.embeddings(face_pixels)
 return yhat[0]
 

    
data = load('Face Dataset.npz',allow_pickle=True)
model = FaceNet() 
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
newTrainX = [];
for face_pixels in trainX:
 embedding = get_embedding(model, face_pixels)
 newTrainX.append(embedding)
newTrainX = asarray(newTrainX)


newTestX = [];
for face_pixels in testX:
 embedding = get_embedding(model, face_pixels)
 newTestX.append(embedding)
newTestX = asarray(newTestX)

savez_compressed('Face Dataset_embeddings.npz', newTrainX, trainy, newTestX, testy)

In [5]:
def predict(trainX,trainY,testX,testY):
    prediction=[];
    for image in testX:
        min_name='';
        min_dist=99999999;
        for i in range(0,len(trainX)):
            dist=np.linalg.norm(trainX[i]-image);
            if(dist<min_dist):
                min_dist=dist;
                min_name=trainY[i];
        prediction.append(min_name);
    return prediction;

In [7]:
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras_facenet import FaceNet
from keras.models import load_model
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC

data = load('Face Dataset_embeddings.npz')
trainX, trainY, testX, testY = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
prediction=predict(trainX, trainY, testX, testY);
for i in range(0,len(testY)):
    if(testY[i]!=prediction[i]):
        print(testY[i],prediction[i]);

Kushal Kumar Mehra Kamesh
Yatish Singhal Himanshu Judge
Sumit Solanki Neil Dahiya
ANKUSH Krishan


In [1]:
from os import listdir
from os.path import isdir
import numpy as np;
from numpy import savez_compressed
from numpy import asarray
from mtcnn.mtcnn import MTCNN
from PIL import Image
import cv2;